## 2D人脸模型回顾

在[之前的学习](https://nbviewer.jupyter.org/github/andyzihaozhang/Face-Recognition-Practice/blob/master/05-06-2020/%E9%98%B6%E6%AE%B5%E6%80%BB%E7%BB%93.ipynb)中，我们熟悉了2D人脸和特征点识别算法。同时练习了使用多种人脸识别模型，在目标追踪算法的辅助下，输入的单张图片或视频流通过预训练模型，绘制人脸对应的选框选，以及标出5点或68点的特征点。但在测试的过程中，当头部的角度，包括在各个轴上的旋转超过一定大小的时候，特征点标注的准确率就会显著降低。同时，一些极端的表情也会干扰特征点的正常标注。由于模型在2D下学习得到信息的有限，上述问题在2D下很难得到有效的解决。

## 3D人脸模型

3D人脸模型在2D特征点识别的基础上，通过获取更多的信息，比如人脸的姿势或者更多的特征点的坐标，来更精确的对目标人脸进行建模和标注。其中，主要学习了3D人脸对准（Alignment）和3D人脸重建（Reconstruction）。

### 3D 人脸对准（Face Alignment）

Reference: [Face Alignment in Full Pose Range: A 3D Total Solution](https://arxiv.org/pdf/1804.01005.pdf)

文章中提到，现有的大多数模型都只针对中等程度的姿态，比如侧脸小于45度，以及所有面部特征点标注的位置都可见。当侧脸达到90度时，会对模型，训练和训练数据这三个方面造成挑战。作者尝试通过将零散的2D特征点变为密集的3DMM（3D Morphable Model）来优化模型。

#### 网络结构
文章的主要成果叫做3D Dense Face Alignment (3DDFA)，主要使用的方法是Cascaded Regression和CNN的结合。

$$p^{k+1} = p^k + Net^k(Fea(I, p^k))$$

其中，p是回归目标（regression object），Fea是图像特征（image feature），Net是CNN网络。网络结构如下图。
![](network_overview.png)

文章提出了两个概念，Pose Adaptive Feature (PAF)和Projected Normalized Coordinate Code (PNCC)。

首先是PAF，作者认为人脸可以被大概看为是一个圆柱体，所以计算圆柱体的坐标，以及在固定的方位角和高度区间里采样64\*64的特征锚点（如下图a）。
![](pac.png)

图(b)是映射在脸上的特征锚点V(p)。$$V(p) = f ∗ Pr ∗ R ∗ (\overline{S} + A_{id}α_{id} + A_{exp}α_{exp}) + t_{2d}$$

括号中的公式代表着3D脸 **S**，通过在平均脸加上形状和表达上的变化得到。**f**是比例常数，**Pr**是直角投射矩阵（orthographic projection matrix），**R**是旋转矩阵（rotation matrix），**t_2d**是转换向量（translation vector）

其次是PNCC，作者将人脸在x、y、z轴上做归一化处理。$$NCC_d = \frac{\overline{S}_d − min(\overline{S}_d)}{max(\overline{S}_d) − min(\overline{S}_d)} \quad (d = x, y, z)$$ 

$$PNCC = Z-Buffer(V_{3d}(p), NCC)$$
这里Z-Buffer可以将映射的3D人脸通过NCC来上色，达到下图的效果。
![](pncc.png)

在每个迭代k，网络会通过PAC和PNCC两条路径训练，最终的特征输出会汇合来预测变量更新。$$\Delta p^k = Net^k(PAF(p^k, I), PNCC(p^k, I))$$

#### 代价函数（cost function)
文章中，作者讨论了四个代价函数（cost function)，分别是Parameter Distance Cost (PDC)，Vertex Distance Cost (VDC)，Weighted Parameter Distance Cost (WPDC)和 Optimized Weighted Parameter Distance Cost (OWPDC)。前两个是基准代价函数，后两个是作者在此基础上提出的代价函数。

PDC可以用来减小groundtruth和当前变量之间的距离。VDC可以用来减小groundtruth和当前3D人脸的顶点距离（vertex distance）。WPDC赋予不同的变量不一样的权重，当高权重变量逐渐接近groundtruth时，该变量的权重会逐渐减小，网络则会开始优化不那么重要的变量，同时保持高权重变量。OWPDC在WPDC的基础上，将所有变量视为一个变量而不是将各个变量分开评估。

#### 应用
依据这篇论文提到的3DDFA模型在PyTorch平台的3D人脸对准应用：https://github.com/cleardusk/3DDFA

![](obama_three_styles.gif)

### 3D人脸重建（Face Reconstruction）

Reference: [Accurate 3D Face Reconstruction with Weakly-Supervised Learning: From Single Image to Image Set](https://arxiv.org/pdf/1903.08527.pdf)

文章提到在训练3D人脸模型时，由于3D人脸的groundtruth数量较少，大多数的groundtruth都是通过合成得到，从而导致不完美的训练集label。为了解决这一问题，作者引入了弱监督学习的概念，试图解决以上的问题。在获得一个可以通过单张图片重建3D人脸的CNN模型后，作者进一步考虑到使用同一个人多张图片重建3D人脸，使得模型的输入可以为任意张图片。
![](framework.png)

本文作者同样使用了3DMM。与上一篇文章不同的是，本文不仅考虑脸部的形状（shape）还考虑脸部的纹理（texture)。$$S = S(α, β) = \overline{S} + B_{id}α + B_{exp}β$$ $$T = T(δ) = \overline{T} + B_tδ$$

除了3D人脸模型，作者还加入了照明模型和相机模型，分别对应上图（a）中的 γ 和 p。其中p由旋转矩阵 R 和转换向量 t 构成，和上一篇文章的人脸映射公式的结构十分接近。

#### 代价函数
1. 图片层面的代价（image-level loss)
    1. 光度代价：比较原图片和重见图片每个像素的密集光度区别（dense photometric discrepancy）$$L_{photo}(x) = \frac{\sum_{i \in M}
A_i \cdot \|I_i − I'_i(x) \|_2 }{\sum_{i \in M} A_i}$$

       这里，i 对应每个像素，A 对应着skin color based attention mask。为了在人脸有旋转、浓密胡子、以及浓妆时有更高的鲁棒性，作者在每个像素上计算了皮肤色彩概率P_i $$A_i =
  \begin{cases}
    1   &\text{if } P_i > 0.5\\
    P_i &\text{otherwise}
  \end{cases}$$
    
    2. 特征点代价：作者用state-of-the-art 3D人脸对准模型检测68个特征点{q_n}并映射至3D人脸，得到{q_n'} $$L_{lan}(x) = \frac{1}{N}\displaystyle\sum_{n=1}^{N} \omega_n \|q_n − q'_n(x)\|^2$$
    

        这里，ω_n是特征点权重。作者在实验中对嘴内部和鼻子等特征点取值为20，其他部位特征点取值为1。


2. 感知层面的代价(perception-level loss)
    $$L_{per}(x) = 1 − \frac{<f(I), f(I'(x))>}{\|f(I)\| \cdot \|f(I'(x))\|}$$
    
    这里，f(·)对应深度特征编码（deep feature encoding）。作者使用了3百万张图片训练了一个FaceNet模型来作为特征提取器。下图为使用和不适用感知层面代价的对比，可以看出，在使用感知层面代价时，面部纹理的对比度更高，更加贴合真实形状。
    ![](perception.png)
    
    
3. 正则化（Regularization）
    $$L_{coef}(x) = \omega_α\|α\|^2 + \omega_β\|β\|^2 + \omega_γ\|γ\|^2$$
    
    为了防止脸部形状和纹理退化，作者对3DMM的系数的使用了常用的代价函数。依据经验，三个 ω 分别设为1.0、0.8和1.7e−3。
    
    $$L_{tex}(x) = \sum_{c \in \{r, g, b\}} var(T_c,R(x))$$
    
    为了减少图片本身阴影的影响，作者以上代价函数来惩罚特征图的变化
    
综上所述，R-Net的代价函数L(x)包括了5个代价函数。其中，2个为图片层面的代价，1个为感知层面的代价，2个为正则化代价。

#### 应用
体现文章中3D人脸重建模型结构的Python应用：https://github.com/microsoft/Deep3DFaceReconstruction
![](reconstruction.gif)

## 总结：从3D人脸到3D人体

* 在引入3D人脸模型后，许多2D时丢失的信息都能够被提取，从而获得更准确的人脸轮廓、特征和纹理。
* 在学习3D人脸对齐和重建的过程中，有很多的研究方法和代价函数设计都可以应用到3D人体对齐和重建的应用中。未来的肺部分割可以不仅限于依据人体骨骼点检测的结果画出肺部区域选框。在不考虑移动端运行速度限制时，使用[这篇文章](http://files.is.tuebingen.mpg.de/classner/up/)提出的模型选出肺部区域似乎是更精确的办法。